In [ ]:
%load_ext tensorboard
%tensorboard --logdir "./runs"

Бейзлайн

In [6]:
!python -W ignore -m main --type eval --wanted_words yes  --dev_every 1 --n_labels 12 --model res8-narrow --input_file honk/model/res8_narrow.pt --no_cuda --data_folder honk/data/speech_dataset

Test accuracy: 0.82, test f1-score: 0.88


Речь со звуками, произведёнными KS-алгоритмом

In [7]:
!python -W ignore -m main --use_noise True --type eval --wanted_words yes  --dev_every 1 --n_labels 12 --model res8-narrow --input_file honk/model/res8_narrow.pt --no_cuda --data_folder honk/data/speech_dataset

Test accuracy: 0.80, test f1-score: 0.86


Пример звуков

In [ ]:
Речь + звуки натренированного KS-алгоритма

In [10]:
!python -W ignore -m main --use_noise True --path_noiser weights/epoch_10.pth True --type eval --wanted_words yes  --dev_every 1 --n_labels 12 --model res8-narrow --input_file honk/model/res8_narrow.pt --no_cuda --data_folder honk/data/speech_dataset

Test accuracy: 0.52, test f1-score: 0.60


Пример звуков

Звуки похожи на результаты авторов статьи

https://www.youtube.com/watch?v=r4XXGDVs0f8&feature=emb_title